# Are users home locations in Vulnerable Locations?

In [2]:
import pandas as pd; import numpy as np; from multiprocessing import Pool, Manager; import numpy as np; import geopandas as gpd
import matplotlib.pyplot as plt; import seaborn as sns
import matplotlib, os, json, sys, time, datetime
from bson import json_util

In [3]:
input_directory  = "/data/chime/geo2/PROCESSED/NJ/AtlanticCity_Stage2/"
output_directory = "/data/chime/geo2/PROCESSED/NJ/AtlanticCity_Stage3/"
zoneGeometry     = "../EvacuationZones/NewJersey/NJ_AtlanticCity.shp"

#### Import the ZoneA Geometry

In [4]:
import fiona, shapely; from osgeo import ogr; from shapely.geometry import mapping, shape
c = fiona.open(zoneGeometry,'r')
pol = c.next(); zone = shape(pol['geometry']).buffer(0); zone.is_valid

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  This is separate from the ipykernel package so we can avoid doing imports until


True

#### Import the user metadata DF (Phasing this out)

In [5]:
_user_meta = pd.read_json(input_directory+'/temporal_clustered_user_meta.json')

In [6]:
_user_meta = _user_meta.sort_index()
_user_meta.head(3)

,home_cluster,home_cluster_coords,tweets,uid,user
0,1.0,"{""coordinates"": [-73.69247457249924, 40.673434...",14231,75153082,Andrewthemark
1,2.0,"{""coordinates"": [-74.40499262545818, 39.386719...",3857,318103926,lanceeryann
2,1.0,"{""coordinates"": [-73.81296028845227, 40.776988...",3778,190488649,giaco1025


## 0. Import all of the individual user dataframes

In [7]:
users_in = sorted(os.listdir(input_directory))
users_in = [x for x in users_in if x != "temporal_clustered_user_meta.json"]
print("Found {0} users in {1}".format(len(users_in), input_directory))

Found 239 users in /data/chime/geo2/PROCESSED/NJ/AtlanticCity_Stage2/


In [8]:
def loader_function(args):
    uFile, path, q = args
    u = json.load(open(path+"/"+uFile,'r'))
    tweets = []
    for t in u['features']:
        if t['geometry']:
            t['properties']['geometry'] = shape(t['geometry'])
        t['properties']['date'] = pd.Timestamp(t['properties']['date'])
        tweets.append(t['properties'])
    q.put(1)
    return gpd.GeoDataFrame(tweets)

In [9]:
#Parallel runtime
p = Pool(30)
m = Manager()
q = m.Queue()

args = [(i, input_directory, q) for i in users_in]
result = p.map_async(loader_function, args)

# monitor loop
while True:
    if result.ready():
        break
    else:
        size = q.qsize()
        sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(size, size/len(args)*100))
        time.sleep(0.5)
sys.stderr.write("\rProcessed: {0}, {1:.3g}%".format(q.qsize(), q.qsize()/len(args)*100))
users = result.get()
p.close()

Processed: 239, 100%%

In [10]:
users.sort( key=lambda x: len(x), reverse=True ) 

`users` is an array of user dataframes. Now find which users have _home locations_ in Zone A

In [11]:
x = users[100]
print(len(x))
x.head(1)

182


,cluster,cluster_center,coords,date,day_cluster,geo_delta,geometry,home_cluster_id,speed,text,time_delta,tweet_id,uid,user
0,-1,null,"[-74.38602445, 40.54273933]",2012-09-02 16:24:44+00:00,9,NaN,POINT (-74.38602444999999 40.54273933),1.0,NaN,@_uptoPAR AC,NaN,242297015298560000,244000867,Cflo412


In [12]:
def get_home_cluster_center(userDF):
    return shape(json.loads(userDF.query('cluster=='+str(userDF.home_cluster_id.values[0])).cluster_center.values[0]))

get_home_cluster_center(users[0]).within(zone)

False

## 1. Determine who's home cluster center is in ZONE

In [13]:
def insideZone(p):
    if p==None:
        return False
    else:
        return p.within(zone)

In [14]:
vuln = []
non_vuln = []
for idx, u in enumerate(users):
    if (insideZone(get_home_cluster_center(u))):
        vuln.append(u)
    else:
        non_vuln.append(u)
    sys.stderr.write("\r"+str(idx+1))
sys.stderr.write("\rDone...")
sys.stderr.write("Identified {0} vulnerable users and {1} non-vulnerable".format(len(vuln),len(non_vuln)))

Done...Identified 37 vulnerable users and 202 non-vulnerable

In [17]:
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

## 2. Write out just the GeoVulnerable, just in case we need them for something later

In [18]:
def safe_mapping(p):
    if p==None or np.isnan(p).any():
        return None
    else:
        return mapping(p)
def safe_json_export(args):
    df, path = args
    df = df.copy()
    uName = df.head(1).user.values[0].lower()
    df['date'] = df['date'].apply(lambda t: datetime.datetime.strftime(t,'%Y-%m-%dT%H:%M:%SZ'))

    clean = df.where((pd.notnull(df)), None)
    geojson = {"type":"FeatureCollection","features":[]}
    for _, row in clean.iterrows():
        geom = safe_mapping(row.geometry)
        feature = {'type':'Feature',
                   'geometry':geom,
                   'properties':row.to_dict()
                    }
        del feature['properties']['geometry']
        geojson['features'].append(feature)
    
    with open(path+"/"+uName+'.geojson','w') as oFile:
        json.dump(geojson, oFile) 

In [16]:
for idx, u in enumerate(vuln):
    safe_json_export((u,output_directory))
    sys.stderr.write("\r{0} processed".format(idx+1))

37 processed

# 3. Prepare for Analysis

All of these users should already exist in a format prepared for visualizing, these users can be pasted into Google Sheets

In [15]:
for uName in sorted([u.user[0].lower() for u in vuln]):
    print(uName)

aclive35
adalicandro
alanjonesbeats
alreadywifedup1
anniexdarling
bellax3
chuck430
chugggss
coachhollywoodp
eazy_sims
elfilosofo92
eyeofh
gacafone
inspired_view
jerodlhoward
jl092479
jockeymajorac
joeythefox323
jpr021
keithpwhite
leeuhbfab
lynd5969
maarieemaariee_
markamccormack
mmmmarguerite
newjerseymma
newjerseypoker
nyr11messier
phantasmosphere
redjazz43
rif_raff13
seanplanesmusic
skinnyminnieee_
summershinystar
tabriahthough
thejkinz
tonyventnor


In [16]:
len(vuln)

37

In [17]:
def good_storm_tweets(userDF):
    return len(userDF.query("date > 201210300000 & date < 201210310000")) 
    
good_data = []
res = []
for idx, u in enumerate(vuln):
    if good_storm_tweets(u) > 2:
        good_data.append(u)
    
    sys.stderr.write("\r"+str(idx))

36

In [18]:
len(good_data)

11

In [23]:
def good_storm_tweets(userDF):
    return len(userDF.query("date > 201210300000 & date < 201210310000")) 
    
good_data = []
res = []
for idx, u in enumerate(vuln):
    if good_storm_tweets(u) > 2:
        good_data.append(u)
    
    sys.stderr.write("\r"+str(idx))

sys.stderr.write("\r"+str(len(good_data)))
for uName in sorted([u.user[0].lower() for u in good_data]):
    print(uName)

23

alanjonesbeats
alreadywifedup1
bellax3
chugggss
leeuhbfab
lynd5969
nyr11messier
redjazz43
rif_raff13
tabriahthough
thejkinz


11

# Create rules file from UID

In [19]:
rules = []
these_rules = []
for idx,u in enumerate(vuln):

    rule = "from:"+u.uid[0]
    these_rules.append(rule)
    
    if idx%25==0 and idx>0:
        rules.append(" OR ".join(these_rules))
        these_rules = []

output = []
for r in rules:
    output.append({"value":r})

with open('../../GNIP/Sandy/NJ_GeoVulnerable_Contextual/rules.json','w') as oFile:
    json.dump(output, oFile)